In [0]:
%cd './gdrive/My Drive/Colab Notebooks/CS263/'     
!pip install cache-magic
import cache_magic
!mkdir .cache
!ln -s './gdrive/My Drive/Colab Notebooks/CS263/.cache' /content/.cache

In [0]:
import pandas as pd 
train = pd.read_csv('./toxic_dataset/train.csv')

In [0]:
def make_final(train):
  train['final'] = 0
  for i in range(train.shape[0]):
    if(train.iloc[i,2] == 1 or train.iloc[i,3] == 1 or train.iloc[i,4] == 1 or train.iloc[i,5] == 1 or train.iloc[i,6] == 1 or train.iloc[i,7] == 1):
      print(i)
      train.loc[i,'final'] = 1
    else:
      train.loc[i,'final'] = 0
  return train

%cache train = make_final(train)

In [0]:
!pip install pytorch-pretrained-bert

import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
text = train['comment_text'][4]
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

In [0]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

In [0]:
# Mark each of the 22 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

In [0]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [0]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

In [0]:
print ("Number of layers:", len(encoded_layers))
layer_i = 0

print ("Number of batches:", len(encoded_layers[layer_i]))
batch_i = 0

print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))

In [0]:
# For the 5th token in our sentence, select its feature values from layer 5.
token_i = 5
layer_i = 11
vec = encoded_layers[layer_i][batch_i][token_i]

# Plot the values as a histogram to show their distribution.
plt.figure(figsize=(10,10))
plt.hist(vec, bins=200)
plt.show()

In [0]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(encoded_layers, dim=0)

token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

In [0]:
token_vecs = encoded_layers[11][0]

sentence_embedding = torch.mean(token_vecs, dim=0)

In [0]:
train_abuse = train.loc[train['final'] == 1, :]
train_clean = train.loc[train['final'] == 0, :]

train_abuse.reset_index(drop=True, inplace=True)
train_clean.reset_index(drop=True, inplace=True)

In [0]:
def make_sentence_embeddings(train):
  embeddings_list = []
  for i in range(train.shape[0]):
    if(i%10 == 0):
      print(i)
    text = train['comment_text'][i]
    marked_text = "[CLS] " + text + " [SEP]"

    tokenized_text = tokenizer.tokenize(marked_text)
    if(len(tokenized_text) > 512):
      continue
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
#    print(tokens_tensor.size(), segments_tensors.size())

    model.eval()
    with torch.no_grad():
      encoded_layers, _ = model(tokens_tensor, segments_tensors)

    token_vecs = encoded_layers[11][0]
    sentence_embedding = torch.mean(token_vecs, dim=0)
    embeddings_list.append(sentence_embedding)
  return embeddings_list

%cache toxic_embedding = make_sentence_embeddings(train_abuse)
%cache clean_embedding = make_sentence_embeddings(train_clean[:20000])  

In [0]:
df_toxic_embedding = pd.DataFrame(toxic_embedding[:1000])
df_clean_embedding = pd.DataFrame(clean_embedding[:1000])

In [0]:
import time
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [0]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results_toxic = tsne.fit_transform(df_toxic_embedding)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [0]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results_clean = tsne.fit_transform(df_clean_embedding)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [0]:
plt.figure(figsize=(16,10))
plt.title("t-SNE of embeddings of toxic vs non-toxic comments")
toxic = plt.scatter(tsne_results_toxic[:,0], tsne_results_toxic[:,1], s = 4, c='red')
clean = plt.scatter(tsne_results_clean[:,0], tsne_results_clean[:,1], s = 4, c='blue')
plt.legend((toxic,clean),
           ('Toxic','Non-Toxic'),
           scatterpoints=1,
           loc='lower left',
           ncol=3,
           fontsize=16)
plt.xlim((-20,20))
plt.ylim((-20,20))